In [137]:
import os
import sys

import sqlite3
import calendar

from datetime import datetime

In [138]:
import nltk
# nltk.download('punkt')
from bs4 import BeautifulSoup
from newspaper import Article
from newscatcher import Newscatcher, describe_url, urls

In [139]:
current_dir = os.getcwd()
news_data_dir = os.path.join(current_dir, 'data')

news_url_dir = os.path.join(news_data_dir, 'news_url')
news_keyword_dir = os.path.join(news_data_dir, 'news_keyword')
news_text_dir = os.path.join(news_data_dir, 'news_text')

In [140]:
today = datetime.today().strftime("%Y-%m-%d")

news_topic = ['tech', 'news', 'business', 'science', 'finance', 'food', 'politics', 'economics', 'travel', 'entertainment', 'music', 'sport', 'world']

news_url = {
    'newyork_times': 'nytimes.com',
    'washington_post':'washingtonpost.com',
    'wallstreet_journal':'wsj.com',
    'cnn':'cnn.com',
    'cnbc':'cnbc.com',
    'financial_times':'ft.com/',
    'reuters':'reuters.com'
}
news_url_list = list(news_url.values())

In [141]:
def crawl_news_data(today, url_list):
    # [source, topic, title, publish_date, link, text]
    # link = news_url, newspaper3k package will load news contents
    # link from newscatcher, text from newspaper3k

    news_base_data = []
    number = 0

    for base_url in url_list:
        
        number += 1
        print(number, base_url)
        
        # topics of base_url
        topics = describe_url(base_url)['topics']
        
        # loop each topics
        try:
            for topic in topics:
                nc = Newscatcher(base_url, topic = topic)
                results = nc.get_news()
                articles = results['articles']

                # base_url - topic pair
                for article in articles:
                    title = article['title']
                    pub_date = article['published']
                    link = article['link']

                    temp = []

                    temp.append(base_url)
                    temp.append(topic)
                    temp.append(title)
                    temp.append(pub_date)
                    temp.append(link)

                    news_base_data.append(temp)

        except:
            print(base_url, topic, title)

    df_news_base_data = pd.DataFrame(news_base_data, columns=['source','topic','title','publish_date','link'])
    df_news_base_data = df_news_base_data.drop_duplicates()
    df_news_base_data.reset_index()
    df_news_base_data.loc[:,'date'] = today

    return df_news_base_data

In [142]:
def load_news_contents(today, df_news_url):
    # text from newspaper3k
    df_news_data_keyword = df_news_url[['source','topic']].copy()
    df_news_data_text = df_news_url[['source','topic']].copy()

    df_news_data_text.loc[:,'date'] = today
    df_news_data_keyword.loc[:,'date'] = today

    try:
        for i in df_news_url.index:
            print(i, df_news_base_data.loc[i,'source'], df_news_url.loc[i,'title'])
            url = df_news_url.loc[i, 'link']
            article = Article(df_news_url.loc[i,'link'], language='en')

            article.download()
            article.parse()
            article.nlp()

            title, keywords, text = article.title, article.keywords, article.text

            keyword_str = ''
            for keyword in keywords:
                keyword_str = keyword_str + keyword + '/'

            df_news_data_text.loc[i ,'title'] = title
            df_news_data_text.loc[i ,'text'] = text
                
            df_news_data_keyword.loc[i ,'title'] = title
            df_news_data_keyword.loc[i ,'keyword'] = keyword_str
    except:
        print(i, df_news_url.loc[i,'source'], df_news_url.loc[i,'title'])
    
    df_news_data_text.reset_index()
    df_news_data_keyword.reset_index()

    return df_news_data_text, df_news_data_keyword

In [143]:
df_news_base_data = crawl_news_data(today, news_url_list)
df_news_data_text, df_news_data_keyword = load_news_contents(today, df_news_base_data)

1 nytimes.com
2 washingtonpost.com

No results found check internet connection or query parameters

washingtonpost.com news How War and Sanctions Make the Ruble Harder to Trade
3 wsj.com

No results found check internet connection or query parameters

wsj.com travel How War and Sanctions Make the Ruble Harder to Trade
4 cnn.com
cnn.com world Women still earn less than men. 6 leaders explain how to fix that
5 cnbc.com
6 ft.com/

No results found check internet connection or query parameters

ft.com/ news Intel pours €30bn into chip manufacturing in Europe
7 reuters.com

No results found check internet connection or query parameters

reuters.com world Intel pours €30bn into chip manufacturing in Europe
0 nytimes.com ‘I Don’t Have the Right to Cry’: Ukrainian Women Share Their Stories of Escape
1 nytimes.com Russia Is Losing Tens of Thousands of Outward-Looking Young Professionals
2 nytimes.com Ukraine Live Updates: Fears Grow for Mariupol as Russia Appears Stalled Elsewhere
3 nytimes.com

In [144]:
df_news_base_data.to_excel(os.path.join(news_url_dir,f'{today}_news_url.xlsx'))
df_news_data_text.to_excel(os.path.join(news_text_dir,f'{today}_news_text.xlsx'))
df_news_data_keyword.to_excel(os.path.join(news_keyword_dir,f'{today}_news_keyword.xlsx'))

In [134]:
df_news_base_data

,source,topic,title,publish_date,link,date
0,nytimes.com,world,‘I Don’t Have the Right to Cry’: Ukrainian Wom...,"Sun, 20 Mar 2022 10:58:31 +0000",https://www.nytimes.com/2022/03/20/world/europ...,2022-03-20
1,nytimes.com,world,Russia Is Losing Tens of Thousands of Outward-...,"Sun, 20 Mar 2022 12:42:34 +0000",https://www.nytimes.com/2022/03/20/world/middl...,2022-03-20
2,nytimes.com,world,Ukraine Live Updates: Fears Grow for Mariupol ...,"Sun, 20 Mar 2022 13:16:23 +0000",https://www.nytimes.com/live/2022/03/20/world/...,2022-03-20
3,nytimes.com,world,"As Omicron Surges and Economy Suffers, China T...","Sun, 20 Mar 2022 13:40:24 +0000",https://www.nytimes.com/2022/03/20/world/asia/...,2022-03-20
4,nytimes.com,world,The Battle for Kyiv Looms as a Long and Bloody...,"Sun, 20 Mar 2022 09:51:06 +0000",https://www.nytimes.com/2022/03/19/world/europ...,2022-03-20
...,...,...,...,...,...,...
625,ft.com/,tech,Foxconn: EV venture is flimsy hedge against Ap...,"Wed, 16 Mar 2022 12:17:56 GMT",https://www.ft.com/content/48dea7eb-62f8-4fb1-...,2022-03-20
626,ft.com/,tech,China takes a key step toward chip self-suffic...,"Wed, 16 Mar 2022 08:26:04 GMT",https://www.ft.com/content/557773a7-0dba-4075-...,2022-03-20
627,ft.com/,tech,Intel: heavy spending is no guarantee of a ret...,"Tue, 15 Mar 2022 18:15:18 GMT",https://www.ft.com/content/c707c0fc-c7d9-4a02-...,2022-03-20
628,ft.com/,tech,Ethereum co-founder hits out at economics of f...,"Tue, 15 Mar 2022 15:00:49 GMT",https://www.ft.com/content/47279272-db0c-4acc-...,2022-03-20


In [135]:
df_news_data_text

,source,topic,date,title,text
0,nytimes.com,world,2022-03-20,‘I Don’t Have the Right to Cry’: Ukrainian Wom...,"We were lucky because my husband’s mother, she..."
1,nytimes.com,world,2022-03-20,Russia Is Losing Tens of Thousands of Outward-...,"YEREVAN, Armenia — At the Lumen cafe in the Ar..."
2,nytimes.com,world,2022-03-20,Ukraine Live Updates: Fears Grow for Mariupol ...,"The Lumen cafe in Yerevan, Armenia, has become..."
3,nytimes.com,world,2022-03-20,"As Omicron Surges and Economy Suffers, China T...",When the coronavirus first swept across China ...
4,nytimes.com,world,2022-03-20,The Battle for Kyiv Looms as a Long and Bloody...,Ukraine’s capital is the biggest prize of all ...
...,...,...,...,...,...
625,ft.com/,tech,2022-03-20,NaN,NaN
626,ft.com/,tech,2022-03-20,NaN,NaN
627,ft.com/,tech,2022-03-20,NaN,NaN
628,ft.com/,tech,2022-03-20,NaN,NaN


In [136]:
df_news_data_keyword

,source,topic,date,title,keyword
0,nytimes.com,world,2022-03-20,‘I Don’t Have the Right to Cry’: Ukrainian Wom...,cry/dont/dagestan/car/women/russians/ukrainian...
1,nytimes.com,world,2022-03-20,Russia Is Losing Tens of Thousands of Outward-...,trying/working/web/war/thought/losing/words/su...
2,nytimes.com,world,2022-03-20,Ukraine Live Updates: Fears Grow for Mariupol ...,appears/left/russians/armenia/updates/york/liv...
3,nytimes.com,world,2022-03-20,"As Omicron Surges and Economy Suffers, China T...",omicron/week/countrys/economy/world/peoples/tw...
4,nytimes.com,world,2022-03-20,The Battle for Kyiv Looms as a Long and Bloody...,battle/conflict/long/kyiv/ukrainian/prize/ukra...
...,...,...,...,...,...
625,ft.com/,tech,2022-03-20,NaN,NaN
626,ft.com/,tech,2022-03-20,NaN,NaN
627,ft.com/,tech,2022-03-20,NaN,NaN
628,ft.com/,tech,2022-03-20,NaN,NaN
